# StatsListReader - e-Stat統計表情報取得ガイド

このノートブックでは、jpy-datareaderライブラリの`StatsListReader`を使用して、日本の政府統計e-Statから統計表の一覧情報を取得します。

## 概要

`StatsListReader`は、e-Stat APIの「統計表情報取得」機能を使用して、以下の情報を取得できます：
- 統計表の基本情報（タイトル、説明、統計表ID等）
- 調査年月、公開年月による絞り込み
- 統計分野や政府統計コードによる検索
- キーワード検索

## 事前準備

1. **e-Stat APIキーの取得**
   - [e-Stat API](https://www.e-stat.go.jp/api/)でアプリケーションIDを取得
   - 環境変数として設定するか、直接指定

2. **必要なライブラリ**
   - `jpy-datareader`
   - `pandas`
   - `python-dotenv` (環境変数管理用)


## 1. ライブラリのインポートと設定

In [ ]:
import os
import sys
from pathlib import Path

import requests
import pandas as pd
from dotenv import load_dotenv

# 日本語表示の設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## 2. StatsListReaderインポート

In [ ]:
from jpy_datareader.estat import StatsListReader

## 3. APIキーの読み込み

In [ ]:
# APIキーの読み込み
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

if not appId:
    print("⚠️ APIキーが設定されていません")
    print("以下の環境変数のいずれかを設定してください:")
    print("- ESTAT_APP_ID")
    print("- E_STAT_APPLICATION_ID")
    print("- ESTAT_APPLICATION_ID")
    # 直接設定する場合（テスト用）
    # appId = "あなたのAPIキーをここに入力"
else:
    print(f"✅ APIキーが正常に読み込まれました: {appId[:10]}...")

## 基本的な使用方法

### 4. 最新の統計表を1件取得

In [ ]:
print("=== 最新の統計表情報（1件）===")
statslist = StatsListReader(api_key=appId, limit=1)
df_latest = statslist.read()

print(f"取得件数: {len(df_latest)}件")
print(f"カラム数: {len(df_latest.columns)}")
print("\n主要なカラム:")
for col in df_latest.columns[:10]:  # 最初の10カラムを表示
    print(f"- {col}")

if len(df_latest) > 0:
    print("\n=== 最新統計表の情報 ===")
    print(f"統計表ID: {df_latest.iloc[0].get('id', 'N/A')}")
    print(f"タイトル: {df_latest.iloc[0].get('STAT_NAME', 'N/A')}")
    print(f"表題: {df_latest.iloc[0].get('TITLE', 'N/A')}")
    print(f"調査年月: {df_latest.iloc[0].get('SURVEY_DATE', 'N/A')}")

### 5. JSON形式での生データ取得

APIの生レスポンスを確認したい場合は`read_json()`を使用します。

In [ ]:
# JSON形式で生データを取得
print("=== JSON形式での統計表情報取得 ===")
statslist = StatsListReader(api_key=appId, limit=1)
json_data = statslist.read_json()

# APIレスポンスの構造を確認
print("APIレスポンスの主要キー:")
for key in json_data.keys():
    print(f"- {key}")

# ステータス情報を確認
result_info = json_data.get('GET_STATS_LIST', {}).get('RESULT', {})
print(f"\nAPI実行結果:")
print(f"- ステータス: {result_info.get('STATUS')}")
print(f"- 実行日時: {result_info.get('DATE')}")

# リーダーオブジェクトに格納されたメタデータを確認
print(f"\nメタデータ情報:")
print(f"- 総件数: {statslist.NUMBER}")
print(f"- 取得範囲: {statslist.FROM_NUMBER} - {statslist.TO_NUMBER}")
print(f"- 次のキー: {statslist.NEXT_KEY}")

## 実践的な検索例

### 6. 国勢調査の統計表を検索

In [ ]:
# 国勢調査（政府統計コード: 00200521）の2020年データを検索
print("=== 国勢調査 2020年の統計表検索 ===")
statslist = StatsListReader(
    api_key=appId,
    statsCode="00200521",  # 国勢調査
    surveyYears=2020,      # 2020年調査
    explanationGetFlg="N", # 説明データは不要
    limit=10               # 最大10件
)
df_census = statslist.read()

print(f"取得件数: {len(df_census)}件")
print(f"総データ件数: {statslist.NUMBER}件")

if len(df_census) > 0:
    print("\n=== 取得した統計表一覧 ===")
    for i, row in df_census.head().iterrows():
        print(f"{i+1}. {row.get('TITLE', 'タイトルなし')}")
        print(f"   ID: {row.get('id', 'N/A')}")
        print(f"   調査年月: {row.get('SURVEY_DATE', 'N/A')}")
        print()

### 7. キーワード検索の活用

In [ ]:
# "人口" AND "東京"で検索
print("=== キーワード検索: '人口' ===")
statslist = StatsListReader(
    api_key=appId,
    searchWord="人口",
    limit=5
)
df_keyword = statslist.read()

print(f"取得件数: {len(df_keyword)}件")

if len(df_keyword) > 0:
    print("\n=== 検索結果 ===")
    for i, row in df_keyword.iterrows():
        title = row.get('TITLE', 'タイトルなし')
        stat_name = row.get('STAT_NAME', 'N/A')
        print(f"{i+1}. {title}")
        print(f"   統計名: {stat_name}")
        print(f"   調査年月: {row.get('SURVEY_DATE', 'N/A')}")
        print()

### 8. 統計分野での絞り込み

In [ ]:
# 統計分野「02」（人口・世帯）で検索
print("=== 統計分野検索: 人口・世帯（分野コード:02）===")
statslist = StatsListReader(
    api_key=appId,
    statsField="02",  # 人口・世帯分野
    surveyYears="202001-202312",  # 2020/1-2023/12範囲
    limit=8
)
df_field = statslist.read()

print(f"取得件数: {len(df_field)}件")

if len(df_field) > 0:
    print("\n=== 人口・世帯分野の統計表 ===")
    # 統計名でグループ化して表示
    grouped = df_field.groupby('STAT_NAME')
    for stat_name, group in grouped:
        print(f"\n📊 {stat_name} ({len(group)}件)")
        for _, row in group.head(3).iterrows():  # 各統計から最大3件表示
            print(f"  - {row.get('TITLE', 'タイトルなし')}")
            print(f"    調査年月: {row.get('SURVEY_DATE', 'N/A')}")

## 高度な使用例

### 9. 更新日付での検索と詳細分析

In [ ]:
# 最近更新された統計表を検索
print("=== 最近更新された統計表（2024年1月以降）===")
statslist = StatsListReader(
    api_key=appId,
    updatedDate="20240101-20241231",  # 2024/1/1-2024/12/31
    searchKind=1,  # 統計情報
    collectArea=1, # 全国データ
    limit=15
)
df_recent = statslist.read()

print(f"取得件数: {len(df_recent)}件")

if len(df_recent) > 0:
    print("\n=== データ分析 ===")
    
    # 統計分野の分布
    if 'STATISTICS_NAME' in df_recent.columns:
        stats_dist = df_recent['STATISTICS_NAME'].value_counts().head()
        print("\n📈 統計名別件数（上位5位）:")
        for stat_name, count in stats_dist.items():
            print(f"  {count}件: {stat_name}")
    
    # 調査年月の分布
    if 'SURVEY_DATE' in df_recent.columns:
        survey_years = df_recent['SURVEY_DATE'].str[:4].value_counts().sort_index()
        print("\n📅 調査年別件数:")
        for year, count in survey_years.items():
            print(f"  {year}年: {count}件")
    
    print("\n=== 最新の統計表（詳細情報）===")
    latest = df_recent.iloc[0]
    print(f"統計表ID: {latest.get('id')}")
    print(f"統計名: {latest.get('STAT_NAME')}")
    print(f"表題: {latest.get('TITLE')}")
    print(f"調査年月: {latest.get('SURVEY_DATE')}")
    print(f"公開年月: {latest.get('OPEN_DATE')}")
    print(f"更新日時: {latest.get('UPDATED_DATE')}")
    
    # データの概要を表示
    if 'MAIN_CATEGORY' in df_recent.columns:
        print(f"主分類: {latest.get('MAIN_CATEGORY')}")
    if 'SUB_CATEGORY' in df_recent.columns:
        print(f"副分類: {latest.get('SUB_CATEGORY')}")

### 10. ページネーション（大量データの取得）

In [ ]:
# 大量データを複数回に分けて取得する例
print("=== ページネーション処理の例 ===")

all_data = []
start_position = None
page_count = 0
max_pages = 3  # 最大3ページまで取得

while page_count < max_pages:
    page_count += 1
    print(f"\n--- ページ {page_count} を取得中 ---")
    
    # StatsListReaderを作成
    params = {
        'api_key': appId,
        'statsField': '02',  # 人口・世帯
        'limit': 5,  # 1ページあたり5件
    }
    
    if start_position:
        params['startPosition'] = start_position
    
    statslist = StatsListReader(**params)
    df_page = statslist.read()
    
    if len(df_page) == 0:
        print("データなし - 処理終了")
        break
    
    all_data.append(df_page)
    print(f"取得件数: {len(df_page)}件")
    print(f"範囲: {statslist.FROM_NUMBER} - {statslist.TO_NUMBER}")
    
    # 次のページのキーを確認
    if statslist.NEXT_KEY:
        start_position = statslist.NEXT_KEY
        print(f"次のキー: {start_position}")
    else:
        print("最後のページに到達")
        break

# 全データを結合
if all_data:
    df_combined = pd.concat(all_data, ignore_index=True)
    print(f"\n=== 結合結果 ===")
    print(f"総取得件数: {len(df_combined)}件")
    print(f"ユニークな統計名: {df_combined['STAT_NAME'].nunique()}種類")

### 11. データの保存とエクスポート

In [ ]:
# 検索結果をCSVファイルに保存
print("=== データのエクスポート ===")

# 人口関連の統計表を取得
statslist = StatsListReader(
    api_key=appId,
    searchWord="人口",
    surveyYears="202001-202312",
    limit=20
)
df_export = statslist.read()

if len(df_export) > 0:
    # 必要な列のみを選択
    columns_to_export = [
        'id', 'STAT_NAME', 'TITLE', 'SURVEY_DATE', 
        'OPEN_DATE', 'UPDATED_DATE'
    ]
    
    # 存在するカラムのみを選択
    available_columns = [col for col in columns_to_export if col in df_export.columns]
    df_selected = df_export[available_columns]
    
    # CSVファイルとして保存
    output_file = "人口統計表一覧.csv"
    df_selected.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"✅ {output_file} に {len(df_selected)}件のデータを保存しました")
    
    # ファイル情報を表示
    file_size = Path(output_file).stat().st_size / 1024  # KB
    print(f"ファイルサイズ: {file_size:.1f} KB")
    
    print("\n=== 保存されたデータの概要 ===")
    print(df_selected.info())
else:
    print("❌ 保存するデータがありません")

### 12. 次のステップ

In [ ]:
# 次のステップの例: 特定の統計表IDでデータを取得する場合
if len(df_export) > 0:
    example_id = df_export.iloc[0]['id']
    print(f"\n=== 次のステップの例 ===")
    print(f"統計表ID '{example_id}' のデータを取得する場合:")
    print(f"""from jpy_datareader.estat import StatsDataReader

data_reader = StatsDataReader(
    api_key=appId,
    statsDataId="{example_id}"
)
df_data = data_reader.read()
print(df_data.head())""")
else:
    print("\n次のステップ: 統計表IDを指定してStatsDataReaderでデータを取得")

## まとめ

### StatsListReaderの主要な機能

1. **基本検索**: `limit`パラメータで件数制限
2. **条件検索**: 
   - `statsCode`: 政府統計コード指定
   - `surveyYears`: 調査年月での絞り込み
   - `searchWord`: キーワード検索（AND/OR/NOT対応）
   - `statsField`: 統計分野での分類
3. **ページネーション**: `startPosition`と`NEXT_KEY`による大量データ取得
4. **データ形式**: DataFrame（`read()`）とJSON（`read_json()`）

### 活用のポイント

- **効率的な検索**: 統計コードや分野コードを活用して目的のデータを絞り込む
- **バッチ処理**: ページネーション機能で大量データを安全に取得
- **データ管理**: 取得したメタデータ（総件数、範囲等）を活用した処理制御

### 次のステップ

統計表IDが特定できたら、`StatsDataReader`を使用して実際の統計データを取得することができます。